# Introdução

Esse caderno tem por objetivo a criação de um modelo básico de treinamento, utilizando o classificador [RandomForest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html), que faça a predição de homologação de arquivamentos de procedimentos enviados à 1A.CAM do MPF.

Esse modelo usará apenas os metadados dos procedimentos, sem fazer nenhum processamento textual.

O objetivo desse modelo é servir como um *baseline* de comparações para implementações futuras.


**Nota**: os dados desse modelo foram recuperados de procedimentos que tiveram suas deliberações realizadas após o dia 02/07/2018, data em que a nova composição tomou posse na 1A.CAM.

# Carga de dados e pré-processamento

Vamos fazer a carga dos dados e fazer um pré-processamento tradicional (remoção de atributos que não interessam, criação de variáveis categóricas etc.)

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd

from sklearn import metrics

In [3]:
PATH = "../data/"
df_original = pd.read_json(f'{PATH}/1A.CAM.homologacao-arquivamento.json')

In [4]:
len(df_original)

5462

In [5]:
df_original.columns

Index(['areaAtuacao', 'classe', 'dataAutuacao', 'dataEntrada', 'homologado',
       'id', 'itemCnmp', 'municipio', 'prioritario', 'procedimento',
       'providenciasExecutadas', 'quantidadeConversoes',
       'quantidadeProvidencias', 'urgente'],
      dtype='object')

In [6]:
df_original.head()

,areaAtuacao,classe,dataAutuacao,dataEntrada,homologado,id,itemCnmp,municipio,prioritario,procedimento,providenciasExecutadas,quantidadeConversoes,quantidadeProvidencias,urgente
0,2,3,"May 16, 2016 12:00:00 AM","Aug 3, 2018 5:38:09 PM",1,71564833,1103,60.0,0,1.10.001.000068/2016-52,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,6,0
1,2,3,"Jul 7, 2016 12:00:00 AM","Jul 25, 2018 7:10:53 PM",1,72574520,1542,1541.0,0,1.11.000.000785/2016-57,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4,12,0
2,2,3,"Apr 25, 2017 12:00:00 AM","Jul 24, 2018 5:30:13 PM",1,77742213,1543,3113.0,0,1.30.001.001754/2017-39,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2,6,0
3,2,3,"Feb 14, 2017 12:00:00 AM","Jul 24, 2018 3:48:22 PM",1,76399468,1726,2650.0,0,1.22.005.000023/2017-16,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",3,5,0
4,2,3,"Jul 9, 2013 12:00:00 AM","Jul 27, 2018 3:12:08 PM",1,47526845,1503,4249.0,0,1.33.005.000326/2013-13,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",6,7,0


In [7]:
df_work = df_original.copy()

In [8]:
# Convertendo strings para data
from datetime import datetime

for index in range(len(df_original)):
    df_work.loc[index, 'dataAutuacao'] =  datetime.strptime(df_work.loc[index, 'dataAutuacao'], '%b %d, %Y %I:%M:%S %p')
    df_work.loc[index, 'dataEntrada'] =  datetime.strptime(df_work.loc[index, 'dataEntrada'], '%b %d, %Y %I:%M:%S %p')

In [9]:
removed_columns = ['id', 'procedimento', 'providenciasExecutadas']
df_work = df_work.drop(columns=removed_columns)
df_work.sample(10)

,areaAtuacao,classe,dataAutuacao,dataEntrada,homologado,itemCnmp,municipio,prioritario,quantidadeConversoes,quantidadeProvidencias,urgente
2582,2,3,2016-09-02 00:00:00,2017-05-09 00:00:00,1,1556,5160.0,0,2,5,0
1930,2,3,2015-10-16 00:00:00,2017-08-30 19:18:02,1,2286,810.0,0,3,12,0
303,2,3,2017-05-31 00:00:00,2018-07-03 16:57:32,1,1521,5051.0,0,3,5,0
4053,2,2,2016-04-12 00:00:00,2016-09-16 00:00:00,1,1507,4852.0,0,2,4,0
1229,2,3,2015-12-10 00:00:00,2018-04-06 11:51:43,1,1542,4838.0,0,2,3,0
2935,2,2,2016-11-04 00:00:00,2017-03-29 00:00:00,1,63,4269.0,0,2,4,0
875,2,3,2016-02-18 00:00:00,2018-05-14 14:14:07,1,56,1541.0,0,2,4,0
81,2,3,2018-03-16 00:00:00,2018-08-02 18:16:23,1,1503,5296.0,0,2,4,0
3070,2,2,2016-09-06 00:00:00,2017-03-07 00:00:00,1,3047,1384.0,0,1,2,0
1852,2,2,2016-09-05 00:00:00,2017-09-04 18:56:55,1,1654,4470.0,0,2,4,0


In [10]:
df_work.describe()

,areaAtuacao,classe,homologado,itemCnmp,municipio,prioritario,quantidadeConversoes,quantidadeProvidencias,urgente
count,5462.000000,5462.000000,5462.000000,5.462000e+03,5457.000000,5462.000000,5462.000000,5462.000000,5462.000000
mean,2.266752,2.537532,0.983706,1.980086e+04,2870.772952,0.006957,2.040278,7.393812,0.006957
std,0.833990,0.539216,0.126617,1.890438e+05,1680.426366,0.083127,1.379018,8.934992,0.083127
min,1.000000,1.000000,0.000000,2.000000e+00,1.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,2.000000,1.000000,1.521000e+03,1301.000000,0.000000,1.000000,3.000000,0.000000
50%,2.000000,3.000000,1.000000,1.582500e+03,3066.000000,0.000000,2.000000,5.000000,0.000000
75%,2.000000,3.000000,1.000000,1.874000e+03,4314.000000,0.000000,3.000000,9.000000,0.000000
max,6.000000,5.000000,1.000000,2.007548e+06,5767.000000,1.000000,11.000000,161.000000,1.000000


In [11]:
# tratando os nulos
df_work.fillna(-1, inplace=True)

In [12]:
len(df_work[df_work['homologado'] == 1]), len(df_work[df_work['homologado'] == 0])

(5373, 89)

### Classes desbalanceadas!!!

Conforme podemos ver acima, as classes desse problema são altamente desbalanceadas - apenas 1.63% do conjunto de dados representam procedimentos que não foram homologados.

Isso, muito provavelmente, causará problemas no treino do modelo. Mas, inicialmente, vamos ignorar isso e seguir com o nosso treino.

In [16]:
removed_columns = ['homologado', 'dataEntrada', 'dataAutuacao']
features = [c for c in df_work.columns if c not in removed_columns]

model = RandomForestClassifier()
model.fit(df_work[features], df_work['homologado'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [17]:
model.score(df_work[features], df_work['homologado'])

0.9972537532039546

In [18]:
from sklearn.model_selection import train_test_split

train, valid = train_test_split(df_work, random_state=42)
train.shape, valid.shape

((4096, 11), (1366, 11))

In [19]:
model.fit(train[features], train['homologado'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [20]:
train_preds = model.predict(train[features])
valid_preds = model.predict(valid[features])

In [21]:
from sklearn.metrics import accuracy_score

(accuracy_score(train['homologado'], train_preds), accuracy_score(valid['homologado'], valid_preds))

(0.997802734375, 0.9809663250366032)

In [22]:
valid_preds

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

### Primeira avaliação

Embora não tenha ocorrido overfitting no treinamento e o *score* final tenha sido alto, cabe utilizar uma outra medida para assegurar a qualidade do modelo. Vamos utilizar a ** *matriz de confusão* **.



In [47]:
from sklearn.metrics import confusion_matrix

confusion_matrix(valid['homologado'], valid_preds)

array([[   0,   22],
       [   4, 1340]], dtype=int64)

In [48]:
confusion_matrix(valid['homologado'], valid_preds).ravel()

array([   0,   22,    4, 1340], dtype=int64)

In [49]:
len(valid[valid['homologado']==1]), len(valid[valid['homologado']==0])

(1344, 22)

Ao utilizar o ravel(), temos uma saída do tipo (tn, fp, fn, tp). De onde vemos que:


- tivemos 0 tn - *true negative*; ou seja, o modelo não conseguiu acertar nenhum dos que não tiveram o arquivamento homologado;
- tivemos 22 fp - *false positive*; 22 que não tiveram o arquivamento homologado e o modelo classificou como homologado;
- tivemos 4 fn - *false negative*; foram homologados e o modelo marcou como não homologados;
- tivemos 1340 tp - *true positive*; foram homologados e o modelo marcou corretamente.

Logo, apesar da acurácia do modelo aparentar ser muito boa (98% no validation set), o **modelo não sabe lidar com os não arquivamentos** - não acertou nenhum dos que realmente não foram homologados. Ou seja dos 22 que não foram homologados, o modelo errou todos e ainda marcou 4 não homologados de forma errada.

Fica claro que somente essas features não ajudam na descoberta - o texto da íntegra é, segundo a área negocial, realmente o mais importante.

Vamos continuar avaliando esse modelo.

## Feature importances

In [57]:
import matplotlib.pyplot as plt

feature_importances = pd.DataFrame(model.feature_importances_, index = train[features].columns, 
                                   columns=['importance']).sort_values('importance', ascending=False)
feature_importances

,importance
municipio,0.349731
itemCnmp,0.343017
quantidadeProvidencias,0.201017
quantidadeConversoes,0.066970
classe,0.028063
areaAtuacao,0.011146
prioritario,0.000031
urgente,0.000025


O modelo está dando muita importância ao município que, considerando o bom senso e um pouco de domínio negocial, não faz sentido - o município não deveria interferir.

Lembrar: criar uma coluna para cada uma das listas de palavras-chaves, indicando ausência ou presença